In [51]:
using XLSX
using LibPQ
using DataFrames

# Define the PostgreSQL database connection parameters
file = open("login_for_sql.txt", "r")
username = chomp(readline(file))
password = chomp(readline(file))
close(file)

conn = LibPQ.Connection("dbname=Symmetria user=$username password=$password")

sql_string = "DROP VIEW IF EXISTS useful_combinations_view;
DROP TABLE IF EXISTS fund,funddata, fundhistory, actions, subfund, fund;

CREATE TABLE fund (
  fund_id SERIAL PRIMARY KEY,
  fund_name TEXT NOT NULL,
  short_name TEXT NULL,
  regex_keyword TEXT NULL
);

CREATE TABLE subfund (
  subfund_id SERIAL PRIMARY KEY,
  fund_id INTEGER NOT NULL REFERENCES fund (fund_id),
  short_name TEXT NULL,
  subfund_type TEXT NULL,
  regex_keyword TEXT NULL,
  subfund_name TEXT NOT NULL
);

CREATE TABLE funddata (
  id SERIAL PRIMARY KEY,
  value NUMERIC NOT NULL,
  fund_id INTEGER NULL REFERENCES fund (fund_id),
  subfund_id INTEGER NULL REFERENCES subfund(subfund_id),
  attribute1 TEXT NOT NULL,
  attribute2 TEXT NULL,
  attribute3 TEXT NULL,
  attribute4 TEXT NULL,
  date TIMESTAMP NULL
);


CREATE TABLE actions (
  action_type TEXT PRIMARY KEY,
  description TEXT NOT NULL
);

CREATE TABLE fundhistory (
  origin_id INTEGER NOT NULL REFERENCES fund (fund_id),
  destination_id INTEGER NULL REFERENCES fund (fund_id),
  date TIMESTAMP NOT NULL,
  action_type TEXT NOT NULL REFERENCES actions (action_type),
  founded TIMESTAMP NULL,
  source TEXT NULL
  --PRIMARY KEY (origin_id, destination_id)
);


/*
CREATE TABLE subfundhistory (
  id SERIAL PRIMARY KEY,
  origin_id INTEGER NOT NULL REFERENCES subfund (subfund_id),
  destination_id INTEGER NOT NULL REFERENCES subfund (subfund_id),
  date TIMESTAMP NOT NULL,
  action_type TEXT NOT NULL REFERENCES actions (actiontype)
);
*/


----- VIEWIÐ
----- VIEWIÐ
DROP VIEW IF EXISTS useful_combinations_view;
CREATE VIEW useful_combinations_view AS
SELECT
    f.fund_id AS fund_id,
    f.fund_name AS fund_name,
    sf.subfund_id AS subfund_id,
    sf.subfund_name AS subfund_name,
    sf.subfund_type AS subfund_type,
    fd.attribute1 AS attribute1,
CASE
	WHEN EXTRACT(year from fd.date) > 2015 and attribute2 = 'Hreinar Fjárfestingartekjur' 
		THEN 'Fjárfestingartekjur'
	WHEN EXTRACT(year from fd.date) > 2015 and attribute2 = 'Fjárfestingartekjur' and attribute3 = 'Fjárfestingargjöld' 
		THEN 'Fjárfestingargjöld'
else fd.attribute2
END AS attribute2,
CASE
    WHEN EXTRACT(year from fd.date) > 2015 and fd.attribute3 = 'Fjárfestingargjöld' THEN '(Samtals) Fjárfestingargjöld'
	else fd.attribute3
END AS attribute3,
    fd.attribute4 AS attribute4,
    to_char(fd.date, 'YYYY-MM-DD HH24:MI:SS') AS date,
    fd.value AS value
FROM
    funddata fd
JOIN
    fund f ON fd.fund_id = f.fund_id
JOIN
    subfund sf ON fd.subfund_id = sf.subfund_id
WHERE
    (fd.attribute1 = 'Rekstur' AND fd.attribute2 = 'Iðgjöld' AND fd.attribute3 = '(Samtals) Iðgjöld')
    OR (fd.attribute1 = 'Rekstur' AND fd.attribute2 = 'Lífeyrir' AND fd.attribute3 = '(Samtals) Lífeyrir')
    OR (fd.attribute1 = 'Rekstur' AND fd.attribute2 = 'Hreinar Fjárfestingartekjur' AND fd.attribute3 = '(Samtals) Hreinar fjárfestingartekjur')
    OR (fd.attribute1 = 'Rekstur' AND fd.attribute2 = 'Fjárfestingartekjur' AND fd.attribute3 = '(Samtals) Fjárfestingartekjur')
    OR (fd.attribute1 = 'Rekstur' AND fd.attribute2 = 'Rekstrarkostnaður' AND fd.attribute3 = '(Samtals) Rekstrarkostnaður')
    OR (fd.attribute1 = 'Efnahagsreikningur' AND fd.attribute2 = '(Samtals) Hrein eign til greiðslu lífeyris')
    OR (fd.attribute1 = 'Efnahagsreikningur' AND fd.attribute2 = 'Eignir' AND fd.attribute3 = '(Samtals) Eignir samtals')
    OR (fd.attribute1 = 'Efnahagsreikningur' AND fd.attribute2 = 'Skuldir' AND fd.attribute3 = '(Samtals) Skuldir samtals')
	
	OR (fd.attribute1 = 'Rekstur' AND fd.attribute2 = 'Fjárfestingargjöld' AND fd.attribute3 = '(Samtals) Fjárfestingargjöld')
	OR (fd.attribute1 = 'Rekstur' AND fd.attribute2 = 'Hreinar Fjárfestingartekjur' AND fd.attribute3 = 'Fjárfestingargjöld')
	
    OR (fd.attribute1 = 'Kennitölur' AND fd.attribute2 = 'Hrein raunávöxtun (%)')
    OR (fd.attribute1 = 'Kennitölur' AND fd.attribute2 = 'Ellilífeyrir (%)')
    OR (fd.attribute1 = 'Kennitölur' AND fd.attribute2 = 'Örorkulífeyrir (%)')
    OR (fd.attribute1 = 'Kennitölur' AND fd.attribute2 = 'Makalífeyrir (%)')
    OR (fd.attribute1 = 'Kennitölur' AND fd.attribute2 = 'Barnalífeyrir (%)')
    OR (fd.attribute1 = 'Kennitölur' AND fd.attribute2 = 'Annar lífeyrir (%)')
    OR (fd.attribute1 = 'Kennitölur' AND fd.attribute2 = 'Stöðugildi')
    OR (fd.attribute1 = 'Kennitölur' AND fd.attribute2 = 'Lífeyrisbyrði')
    OR (fd.attribute1 = 'Kennitölur' AND fd.attribute2 = 'Lífeyrisbyrði (%)')
    OR (fd.attribute1 = 'Kennitölur' AND fd.attribute2 = 'Fjárfestingargjöld (bein og áætluð/reiknuð)' AND fd.attribute3 = 'Hlutfall fjárfestingargjalda alls af heildareignum (%)')
    OR (fd.attribute1 = 'Kennitölur' AND fd.attribute2 = 'Skrifstofu og stjórnunarkostnaður (alls) í % af meðalstöðu eigna')
    OR (fd.attribute1 = 'Lífeyrisþegar' AND fd.attribute2 ='Örorku')
    OR (fd.attribute1 = 'Framtíðariðgjöld')
    OR (fd.attribute1 = 'Ellilífeyrir á mán (þús. kr.)' AND fd.attribute2 ='Karlar')
    OR (fd.attribute1 = 'Samtals sjóðfélagar og lífeyrisþegar')
    OR (fd.attribute1 = 'Iðgjöld ársins (þús. kr.)' AND fd.attribute2 ='Karlar')
    OR (fd.attribute1 = 'Fjöldi ellilífeyrisþega' AND fd.attribute2 ='Konur')
    OR (fd.attribute1 = 'Óvirkir sjóðfélagar')
    OR (fd.attribute1 = 'Heildarstaða %' AND fd.attribute2 ='NaN')
    OR (fd.attribute1 = 'Lífeyrisþegar' AND fd.attribute2 ='Elli')
    OR (fd.attribute1 = 'Iðgjöld ársins (þús. kr.)' AND fd.attribute2 ='Konur')
    OR (fd.attribute1 = 'Áætlaðar launagr.' AND fd.attribute2 ='Konur')
    OR (fd.attribute1 = 'Eignir í tryggingafræðilegu mati')
    OR (fd.attribute1 = 'Heildarstaða samtals')
    OR (fd.attribute1 = 'Áætlað iðgjöld %')
    OR (fd.attribute1 = 'Framtíðarstaða %')
    OR (fd.attribute1 = 'Ellilífeyrir á mán (þús. kr.)' AND fd.attribute2 ='Konur')
    OR (fd.attribute1 = 'Fjöldi iðgjaldagreiðenda' AND fd.attribute2 ='Karlar')
    OR (fd.attribute1 = 'Heildarskuldbindingar' AND fd.attribute2 ='NaN')
    OR (fd.attribute1 = 'Virkir sjóðfélagar')
    OR (fd.attribute1 = 'Fjöldi iðgjaldagreiðenda' AND fd.attribute2 ='Konur')
    OR (fd.attribute1 = 'Heildareignir')
    OR (fd.attribute1 = 'Áfallnar skuldbindingar')
    OR (fd.attribute1 = 'Áfallin staða')
    OR (fd.attribute1 = 'Lífeyrisþegar' AND fd.attribute2 ='Barna')
    OR (fd.attribute1 = 'Lífeyrisþegar' AND fd.attribute2 ='Maka')
    OR (fd.attribute1 = 'Fjöldi ellilífeyrisþega' AND fd.attribute2 ='Karlar')
    OR (fd.attribute1 = 'Framtíðarskuldbinding')
    OR (fd.attribute1 = 'Áfallin staða %')
    OR (fd.attribute1 = 'Framtíðarstaða')
    OR (fd.attribute1 = 'Áætlaðar launagr.' AND fd.attribute2 ='Karlar')
    OR (fd.attribute1 = 'Samtals sjóðfélagar');




-- ná öllum distinct attributeum í viewinu, ætti að vera 50
SELECT DISTINCT
  attribute1,
  attribute2,
  attribute3,
  attribute4
FROM
  useful_combinations_view;

---- Ná öllum distinct attributeum, og finna hvaða ár það er í

WITH attribute_combinations AS (
    SELECT DISTINCT
        attribute1,
        attribute2,
        attribute3,
        attribute4
    FROM
        funddata
    ORDER BY
        attribute1,
        attribute2,
        attribute3,
        attribute4
),

years_combinations AS (
    SELECT
        attribute1,
        attribute2,
        attribute3,
        attribute4,
        ARRAY_AGG(DISTINCT EXTRACT(YEAR FROM date)::integer) AS years
    FROM
        funddata
    GROUP BY
        attribute1,
        attribute2,
        attribute3,
        attribute4
)

SELECT
    ac.attribute1,
    ac.attribute2,
    ac.attribute3,
    ac.attribute4,
    yc.years AS \"occurs in\"
FROM
    attribute_combinations ac
JOIN
    years_combinations yc ON ac.attribute1 = yc.attribute1
                            AND ac.attribute2 = yc.attribute2
                            AND ac.attribute3 = yc.attribute3
                            AND ac.attribute4 = yc.attribute4
ORDER BY
    ac.attribute1,
    ac.attribute2,
    ac.attribute3,
    ac.attribute4;


---
"
result = execute(conn,sql_string) |> DataFrame
println("Reset á gagnagrunninum : $result")   

Reset á gagnagrunninum : 0×5 DataFrame
 Row │ attribute1  attribute2  attribute3  attribute4  occurs in
     │ String?     String?     String?     String?     Abstract…?
─────┴────────────────────────────────────────────────────────────


NOTICE:  view "useful_combinations_view" does not exist, skipping


In [52]:
using XLSX
using LibPQ
using DataFrames

# Define the PostgreSQL database connection parameters
file = open("login_for_sql.txt", "r")
username = chomp(readline(file))
password = chomp(readline(file))
close(file)

conn = LibPQ.Connection("dbname=Symmetria user=$username password=$password")

sql_string = "INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Allianz','Allianz','.*(A|a)llianz.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Almenni lífeyrissjóðurinn','Almenni','.*Almenni.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Almennurlífeyrissj. VÍB','AVÍB','.*VÍB.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Arion banki hf.','Arion','.*(A|a)rion.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Bayern','Bayern','(b|B)ayern');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Birta lífeyrissjóður','Birta','.*Birta.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Brú Lífeyrissjóður starfsmanna sveitarfélaga','Brú','.*(B|b)rú.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Eftirlaunasj. Landsbanka og Seðlabanka','Landsbanka og Seðlabanka','Eftirlaunasj.*\\sLandsb(anka|\\.) (og|&) Seðlabanka');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Eftirlaunasj. Sláturfél. Suðurlands','Eftirlaunasj. Sláturfél. Suðurlands','Eftirlaunasj.*(Sláturf(él|.)).*(Suðurlands).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Eftirlaunasj. starfsm. Íslandsb.','Eftirlaunasj. Íslandsb.','Eftirlaunasj(\\.|óður)(\\s|)(s|S)tarfsm(\\.|anna|m\\.)(\\s|)Íslandsb(\\.|anka)(\\s(h|H)f\\.|).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Eftirlaunasj. stm. Glitnis banka','Glitnir','.*Glitn.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Eftirlaunasj. stm. Hafnarfjarðk.','Eftirlaunasj. Hafnarfjarðarkaupstaðar','Eftirlaunasj.*(Hafnarfj).*(kaupst\\.|kaupstaðar|arðk\\.|arðark\\.)');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Eftirlaunasj. stm. OLÍS','OLÍS','Eftirlaunasj.*(|\\n)(Olí|OLÍ).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Eftirlaunasjóður FÍA','EFÍA','(EFÍA|Eftir(-|)launa(-|-\\s|)sj(\\.|\\s|óður)(\\s|\\s\\s|)(atvinnuflugmanna|FÍA))');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Eftirlaunasjóður Reykjanesbæjar','Eftirlaunasj. Reykjanesbæjar','Eftirlaunasj.*Reykjanes(-|-\\s|)bæjar');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Eftirlaunasjóður slökkviliðsmanna á Keflavíkurflugvelli','Eftirlaunasj. Slökkviliðsmanna á Kef','.*Eftirlaunasj.*(\\s|)slökkvil.* á(\\s|)Keflav.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Eftirlaunasjóður starfsmanna Útvegsbanka Íslands','Eftirlaunasj. Útvegsbanka','Eftirl.*\\s*(starfsm|stm).*\\s*(Ú|ú)tvegsb.*\\s*Ísl.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Festa - lífeyrissjóður','Festa','.*Festa.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Frjálsi lífeyrissjóðurinn','Frjálsi','.*Frjálsi.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Gildi - lífeyrissjóður','Gildi','.*(G|g)ildi.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Íslandsbanki hf.','Íslandsbanki','.*(Í|í)slandsbanki.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Íslenski lífeyrissjóðurinn','Íslenski','.*Íslenski.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Kjölur lífeyrissjóður','Kjölur','(Kjölur lífeyris).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Kvika banki hf.','Kvika','.*(K|k)vika.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Landsbankinn hf.','Landsbankinn','.*(L|l)andsbankinn.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður lækna','Lífeyrissj. Lækna','Lífeyris(-sjóður|sjóður|sj)(\\s|\\.|\\.\\s)(l|L)ækna.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Suðurlands','Lífeyrissj. Suðurlands','Lífeyris(-|)(sj\\.|sjóður )(\\n|\\s|)Suðurlands');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóðurinn Skjöldur','Lífeyrissj. Skjöldur','(Lífeyris).*(sjóðurinn).*(Skjöldur)');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóðurinn Eining','Eining','.*Eining.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóðurinn Hlíf','Hlíf','.*Hlíf.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissj. Akraneskaupstaðar','Lífeyrissj. Akraneskaupstaðar','.*(Akranes(-|-\\s|)kaupst).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissj. Bolungarvíkur','Lífeyrissj. Bolungarvíkur','(L|l)ífeyrissj.*(\\.|\\s)(Bolungar).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissj. Eimskipa-félags Ísl.','Lífeyrissj. Eimskipafélag','(Lífeyris).*(sj).*(Eimskipa).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissj. Fél. ísl. stjstarfsm. á Keflvflugvelli','Lífeyrissj. Félag starfsmanna á Kef flugvelli','Lífeyrissj.*ísl.*stjst.* á Kefl.{0,1}v.{0,4}flugv.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissj. Neskaupstaðar','Lífeyrissj. Neskaupstaðar','.*((\\.|\\s)Nes(-|)kaup).*(\\.|staðar)');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissj. stm. Húsavíkurk.','Lífeyrissj. Húsavíkurkaupstaðar','.*(Húsa(-|-\\s|)víkur(k|k\\.|-|))(\\.|aupstaðar|bæjar|).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissj. stm. Reykjavíkur apóteks','Lífeyrissj. Reykjavíkurapóteks','.*Lífeyris(-|)(sj\\.|sjóður )(\\s|)(stm\\.|starfsm\\.|starfsmanna)\\sR(eykjavíkur|vík\\.)(-|-\\s|\\s|)apóteks.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissj. stm. Vestm.eyjab.','Lífeyrissj. Vestmannaeyjabæjar','Lífeyrissj(\\.|óður\\s)(\\s|)(stm|starfsmanna|starfsm\\.)(\\.\\s|\\s|)Vestm.*eyja(\\s|b\\.|bæjar|-bæjar)');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissj. arkitekta og tæknifr.','Lífeyrissj. Arkitekta og tæknifræðinga','.*(A|a)rkitekta.*(T|t)æknifr.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissj. K.E.A.','K.E.A','.*K\\.E\\.A\\..*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissj.verkalfél.Norðurl. v','Lífeyrissj. Verkalfél. Norðurl. V','.*Líf.*(V|v)erkal.*fél\\.(\\sá\\s|\\s|)Norðurl\\. v.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður alþingismanna','Lífeyrissj. Alþingismanna','Lífeyrissj.* alþingis(-\\s|)manna.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Arkitekta','Lífeyrissj. Arkitekta','Lífeyris(-|-\\s|)sjóður Arkitekta');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Austurlands','Lífeyrissj. Austurlands','(L|l)ífeyris(-|)sj(\\.|óður )(\\s|)(Austur(-|-\\s|)lands).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður bænda','Lífeyrissj. Bænda','.*(B|b)ænda.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður bankamanna','Lífeyrissj. Bankamanna','.*Lífeyrissj.*(b|B)anka(-|-\\s|)m.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður blaðamanna','Lífeyrissj. Blaðamanna','.*(B|b)laða(-\\s|)manna.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Félags íslenskra leikara','Lífeyrissj. Félags íslenska leikara','(l|L)ífeyrissj.*(F|f)élags.*ísl.*leikara');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Flugvirkjafél. Íslands','Lífeyrissj. Flugvirkjafélag','(L|l)ífeyrissj.*(\\.|\\s)(Flugvirkja(-f|fé|f.)).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður hjúkrunarfræðinga','Lífeyrissj. Hjúkrunarfræðinga','.*(L|l)íf.+(H|h)júkrunar(-|-\\s|)fræðinga.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Iðnaðarmannafél. Suðurnesja','Lífeyrissj. Iðnaðarmannafél. Suðurnesja','Lífeyrissj.*Iðnaðarm.*Suðurnesja');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður leigubifreiðastjóra','Lífeyrissj. Leigubifreiðastjóra','.*(L|l)eigubifr.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Mjólkursamsölunnar','Lífeyrissj. Mjólkursamsölunnar','(L|l)ífeyrissj.*(\\.|\\s).*(Mjólkur).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Norðurlands','Lífeyrissj. Norðurlands','(L|l)ífeyrissj.*(\\.|\\s)(Norður(-|-\\s|)(lands|))');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður ráðherra','Lífeyrissj. Ráðherra','Lífeyrissj.* ráðherra');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Rangæinga','Lífeyrissj. Rangæinga','Lífeyris(-|)sj(\\.|óður\\s)(\\s|)Rang(\\.|æinga)');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður sjómanna','Lífeyrissj. Sjómanna','.*(S|s)jómanna.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður starfsm. Sjóvátryggingafél. ísl.','Lífeyrissj. Sjóvátryggingafél','.*(S|s)jóvá.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður starfsmanna Akureyrarbæjar','Lífeyrissj. Akureyrarbæjar','Lífeyrissj(óður|\\.\\s|\\.)(\\s|\\.|)(stm\\.|starfsm)(anna\\s|\\s|\\.\\s)Akur(-|-\\s|)eyrar(b\\.|bæjar)');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður starfsmanna Búnaðarbanka Íslands','Lífeyrissj. Búnaðarbanka','.*(B|b)únaðar.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður starfsmanna Kópavogsbæjar','Lífeyrissj. Kópavogsbæjar','.*Lífeyrissj(óður|\\.)(\\s|)(stm\\.|starfsm\\.|starfsmanna)\\sKópa(-|-\\s|)vogs(b\\.|bæjar|kaupst).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður starfsmanna Reykjavíkurborgar','Lífeyrissj. Reykjavíkurborgar','.*Lífeyris(sjóður|-sjóður|sj\\.)(\\s|\\.|)((S|s)tm\\.\\s|(S|s)tarfsmanna|(S|s)tarfsm\\.)(\\s|)Reykjav(íkurb\\.|b\\.|íkurborgar|\\.).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður starfsmanna ríkisins','Lífeyrissj. ríkisins','(Lífeyrissj.*(starfsm(\\.|anna)|\\.(\\s|)st\\.|\\.\\sst\\.m\\.|stm\\.)\\s(R|r)íkisins|.*LSR.*)');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður starfsmanna sveitarfélaga','Lífeyrissj. Sveitarfélaga','Lífeyriss(s|)j(óður|\\.)(\\s|)(s|S)t.*(\\s|\\.|)(s|S)veitarfél(\\.|aga)');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður stm. Áburðarverks','Lífeyrissj. Áburðarverksmiðju','.*(Áburðarv(.|e)).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Suðurnesja','Lífeyrissj. Suðurnesja','Lífeyrissj(\\.|óður)(\\s|)Suður(-|-\\s|)nesja');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Tæknifræðingafélags Íslands','Lífeyrissj. Tæknifræðingafélags','Lífeyrissj.*Tæknifr.* Íslands');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Tannlæknafél Ísl','Lífeyrissj. Tannlæknafélags','.*(Tannl|tannl).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður verkafólks í Grindavík','Lífeyrissj. Verkafólks í Grindavík','.*Grindavík.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður verkalýðsfélaga á Suðurlandi','Lífeyrissj. Verkalýðsfélaga á Suðurlandi','Lífeyrissj.*verkal.* á Suðurl(andi|\\.)');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður verkfræðinga','Lífsverk','.*(Lífeyrissj).*((V|v)erk.*)fr.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður verslunarmanna','Lífeyrissj. Verslunarmanna','.*((L|l)íf[a-z. óð]*Versl|(L|l)íf[a-z. óð]*versl).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Vestfirðinga','Lífeyrissj. Vestfirðinga','Lífeyrissj(óður|\\.)((\\s|)Vest(-\\s|-\\s\\s|-|)firðinga)');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Vestmannaeyja','Lífeyrissj. Vestmannaeyja','.*Lífeyrissj(\\.|óður)(\\s|)Vestm(\\.|(anna(-|-\\s|)eyja)).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóður Vesturlands','Lífeyrissj. Vesturlands','.*Lífeyris(-|)(sj\\.|sjóður)(\\s|)Vestur(-|-\\s|)lands.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóðurinn Framsýn','Framsýn','.*Framsýn.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífeyrissjóðurinn Lífiðn','Lífeyrissj. Lífiðn','(L|l)ífey.*(\\.|\\s)(Lífiðn)+');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífsval','Lífsval','.*Lífsval.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Lífsverk lífeyrissjóður','Lífsverk','.*(L|l)ífsverk.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Sameinaði lífeyrissjóðurinn','Sameinaði','Sameinaði(\\s|)lífeyris(-|-\\s|)sjóðurinn');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Samvinnulífeyrissjóðurinn','Samvinnulífeyrissj.','.*(S|s)amvinnu(-|).*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Séreignalífeyrissjóðurinn','Séreignalífeyrissj.','.*Séreigna.*lífeyris.*sjóðurinn.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Söfnunarsjóður lífeyrisréttinda','Söfnunarsj. Lífeyrisréttinda','(S|s)öfnunarsj(\\.\\s|\\.|óður\\s)(lífeyris(-|)réttinda|)');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Stafir lífeyrissjóður','Stafir','.*Stafir.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Stapi lífeyrissjóður','Stapi','.*(S|s)tapi.*');
INSERT INTO public.fund(fund_name, short_name, regex_keyword)VALUES ('Tryggingasjóður lækna','Tryggingasj. Lækna','.*Trygginga.*sjóður.*lækna.*');"

result = execute(conn,sql_string) |> DataFrame
println("Niðurstaða insertsins : $result")    

Niðurstaða insertsins : 0×0 DataFrame


In [54]:
using XLSX
using LibPQ
using DataFrames

regex_key = [r".*(A|a)llianz.*",r".*Almenni.*",r".*VÍB.*",r".*(A|a)rion.*",r"(b|B)ayern",r".*Birta.*",r".*(B|b)rú.*",r"Eftirlaunasj.*\sLandsb(anka|\.) (og|&) Seðlabanka",r"Eftirlaunasj.*(Sláturf(él|.)).*(Suðurlands).*",r"Eftirlaunasj(\.|óður)(\s|)(s|S)tarfsm(\.|anna|m\.)(\s|)Íslandsb(\.|anka)(\s(h|H)f\.|).*",r".*Glitn.*",r"Eftirlaunasj.*(Hafnarfj).*(kaupst\.|kaupstaðar|arðk\.|arðark\.)",r"Eftirlaunasj.*(|\n)(Olí|OLÍ).*",r"(EFÍA|Eftir(-|)launa(-|-\s|)sj(\.|\s|óður)(\s|\s\s|)(atvinnuflugmanna|FÍA))",r"Eftirlaunasj.*Reykjanes(-|-\s|)bæjar",r".*Eftirlaunasj.*(\s|)slökkvil.* á(\s|)Keflav.*",r"Eftirl.*\s*(starfsm|stm).*\s*(Ú|ú)tvegsb.*\s*Ísl.*",r".*Festa.*",r".*Frjálsi.*",r".*(G|g)ildi.*",r".*(Í|í)slandsbanki.*",r".*Íslenski.*",r"(Kjölur lífeyris).*",r".*(K|k)vika.*",r".*(L|l)andsbankinn.*",r"Lífeyris(-sjóður|sjóður|sj)(\s|\.|\.\s)(l|L)ækna.*",r"Lífeyris(-|)(sj\.|sjóður )(\n|\s|)Suðurlands",r"(Lífeyris).*(sjóðurinn).*(Skjöldur)",r".*Eining.*",r".*Hlíf.*",r".*(Akranes(-|-\s|)kaupst).*",r"(L|l)ífeyrissj.*(\.|\s)(Bolungar).*",r"(Lífeyris).*(sj).*(Eimskipa).*",r"Lífeyrissj.*ísl.*stjst.* á Kefl.{0,1}v.{0,4}flugv.*",r".*((\.|\s)Nes(-|)kaup).*(\.|staðar)",r".*(Húsa(-|-\s|)víkur(k|k\.|-|))(\.|aupstaðar|bæjar|).*",r".*Lífeyris(-|)(sj\.|sjóður )(\s|)(stm\.|starfsm\.|starfsmanna)\sR(eykjavíkur|vík\.)(-|-\s|\s|)apóteks.*",r"Lífeyrissj(\.|óður\s)(\s|)(stm|starfsmanna|starfsm\.)(\.\s|\s|)Vestm.*eyja(\s|b\.|bæjar|-bæjar)",r".*(A|a)rkitekta.*(T|t)æknifr.*",r".*K\.E\.A\..*",r".*Líf.*(V|v)erkal.*fél\.(\sá\s|\s|)Norðurl\. v.*",r"Lífeyrissj.* alþingis(-\s|)manna.*",r"Lífeyris(-|-\s|)sjóður Arkitekta",r"(L|l)ífeyris(-|)sj(\.|óður )(\s|)(Austur(-|-\s|)lands).*",r".*(B|b)ænda.*",r".*Lífeyrissj.*(b|B)anka(-|-\s|)m.*",r".*(B|b)laða(-\s|)manna.*",r"(l|L)ífeyrissj.*(F|f)élags.*ísl.*leikara",r"(L|l)ífeyrissj.*(\.|\s)(Flugvirkja(-f|fé|f.)).*",r".*(L|l)íf.+(H|h)júkrunar(-|-\s|)fræðinga.*",r"Lífeyrissj.*Iðnaðarm.*Suðurnesja",r".*(L|l)eigubifr.*",r"(L|l)ífeyrissj.*(\.|\s).*(Mjólkur).*",r"(L|l)ífeyrissj.*(\.|\s)(Norður(-|-\s|)(lands|))",r"Lífeyrissj.* ráðherra",r"Lífeyris(-|)sj(\.|óður\s)(\s|)Rang(\.|æinga)",r".*(S|s)jómanna.*",r".*(S|s)jóvá.*",r"Lífeyrissj(óður|\.\s|\.)(\s|\.|)(stm\.|starfsm)(anna\s|\s|\.\s)Akur(-|-\s|)eyrar(b\.|bæjar)",r".*(B|b)únaðar.*",r".*Lífeyrissj(óður|\.)(\s|)(stm\.|starfsm\.|starfsmanna)\sKópa(-|-\s|)vogs(b\.|bæjar|kaupst).*",r".*Lífeyris(sjóður|-sjóður|sj\.)(\s|\.|)((S|s)tm\.\s|(S|s)tarfsmanna|(S|s)tarfsm\.)(\s|)Reykjav(íkurb\.|b\.|íkurborgar|\.).*",r"(Lífeyrissj.*(starfsm(\.|anna)|\.(\s|)st\.|\.\sst\.m\.|stm\.)\s(R|r)íkisins|.*LSR.*)",r"Lífeyriss(s|)j(óður|\.)(\s|)(s|S)t.*(\s|\.|)(s|S)veitarfél(\.|aga)",r".*(Áburðarv(.|e)).*",r"Lífeyrissj(\.|óður)(\s|)Suður(-|-\s|)nesja",r"Lífeyrissj.*Tæknifr.* Íslands",r".*(Tannl|tannl).*",r".*Grindavík.*",r"Lífeyrissj.*verkal.* á Suðurl(andi|\.)",r".*(Lífeyrissj).*((V|v)erk.*)fr.*",r".*((L|l)íf[a-z. óð]*Versl|(L|l)íf[a-z. óð]*versl).*",r"Lífeyrissj(óður|\.)((\s|)Vest(-\s|-\s\s|-|)firðinga)",r".*Lífeyrissj(\.|óður)(\s|)Vestm(\.|(anna(-|-\s|)eyja)).*",r".*Lífeyris(-|)(sj\.|sjóður)(\s|)Vestur(-|-\s|)lands.*",r".*Framsýn.*",r"(L|l)ífey.*(\.|\s)(Lífiðn)+",r".*Lífsval.*",r".*(L|l)ífsverk.*",r"Sameinaði(\s|)lífeyris(-|-\s|)sjóðurinn",r".*(S|s)amvinnu(-|).*",r".*Séreigna.*lífeyris.*sjóðurinn.*",r"(S|s)öfnunarsj(\.\s|\.|óður\s)(lífeyris(-|)réttinda|)",r".*Stafir.*",r".*(S|s)tapi.*",r".*Trygginga.*sjóður.*lækna.*",]

global nullcounter = 0

println(".")
# Define the PostgreSQL database connection parameters
file = open("login_for_sql.txt", "r")
username = chomp(readline(file))
password = chomp(readline(file))
close(file)
println("..")

conn = LibPQ.Connection("dbname=Symmetria user=$username password=$password")
println("...")
# Define the directory containing the Excel files
dir_path = "./skjölin frá Birgi/Files/"

filename = "unique_attribute_values.xlsx"
filepath = joinpath(dir_path, filename)
attribute_map = DataFrame(XLSX.readtable(filepath, "Unique Attribute Values", header = false))
println("..")

filename = "unique_subfundsV3.xlsx"
filepath = joinpath(dir_path, filename)
subfund_map = DataFrame(XLSX.readtable(filepath, "Unique Subfunds", header = false))
println(".")


function insert_standardform(sheet::DataFrame, year::Int64)
    # Loop through each row in the value matrix and insert the data into the PostgreSQL database
    for row in 4:size(sheet, 1)
        for col in 6:size(sheet, 2)
            # Get the relevant data for the current row and column
            fund_name = sheet[1, col]
            subfund_name = subfund_mapping(sheet[2, col])

            value = sheet[row, col]
            
            date = sheet[1, 2]
            
            attribute1 = sheet[row, 1]
            attribute2 = sheet[row, 2]
            attribute3 = sheet[row, 3]
            attribute4 = sheet[row, 4]
            
            if isnothing(value) || ismissing(attribute1) || value == "-"
                value = "NULL"
            end
            
            if typeof(value) == String && value != "NULL"
                value = replace(value, "*" => "")
                
                if year < 2006 && occursin("%",value)
                    
                    value = replace(value, "%" => "")
                    
                    if occursin(",",value)
                        value = replace(value, "," => ".")
                        value = parse(Float64, value)/100
                        value = string(value)
                    end
                else
                    if occursin(",",value)
                        value = replace(value, "," => ".")
                    end
                end
                
                if value != ""
                    value = parse(Float64, value)
                else
                    value = "NULL"
                end
            end
            
            
            if isnothing(attribute1) || ismissing(attribute1)
                attribute1 = "NULL"
            else
                attribute1 = "'" * attribute_mapping(attribute1) * "'"
            end
            
            if isnothing(attribute2) || ismissing(attribute2)
                attribute2 = "NULL"
            else
                 attribute2 = "'" * attribute_mapping(attribute2) * "'"
            end
            
            
            if isnothing(attribute3) || ismissing(attribute3)
                attribute3 = "NULL"
            else
                 attribute3 = "'" * attribute_mapping(attribute3) * "'"
            end
            
            if isnothing(attribute4) || ismissing(attribute4) 
                attribute4 = "NULL"
            else
                 attribute4 = "'" * attribute_mapping(attribute4) * "'"
            end
            
            
            if year <= 2005 && attribute1 == "Kennitölur" && (attribute2 == "Hrein raunávöxtun (%)" || attribute2 == "Meðalraunávöxtun s.l. 5 ára (%)")
                value = value*100
            end
            
            #println("pre $fund_name")
            reg_string = ""
            fund_id = -100
            for reg in regex_key
                if !isnothing(match(reg, fund_name))
                    reg_string = string(reg)[3:end-1]
                    result = execute(conn, "SELECT fund_id FROM fund WHERE regex_keyword = '$reg_string' or short_name = '$fund_name' or fund_name = '$fund_name'") |> DataFrame
                    if size(result,1) == 0
                        error("Fann ekkert fund_id því enginn sjóður með regexið : $reg_string")
                    end
                    fund_id = result[1,1]
                    break
                end
            end
            
            if fund_id == -100
                error("no fund_id found for : $fund_name")
            end
            
            subfund_id = ""
            
            if subfund_id != "NULL"
                result = execute(conn, "SELECT subfund_id FROM subfund WHERE fund_id = $fund_id and subfund_name = '$subfund_name'") |> DataFrame
                if size(result,1) == 0
                    error("Fann ekkert subfund_id, idið á fund er $fund_id, nafn $fund_name, subfund $subfund_name")
                end
                subfund_id = result[1,1]
            end
            
            # Insert the data into the PostgreSQL database
            if value != "NULL"  && value != "null" && value != "Null"
                sql = "INSERT INTO public.funddata (value, fund_id, subfund_id, attribute1, attribute2, attribute3, date, attribute4) VALUES ($value, $fund_id, $subfund_id, $attribute1, $attribute2, $attribute3, '$date', $attribute4);"
                execute(conn, sql)
            else
                global nullcounter += 1
            end
        end
    end
end
println("..")
function attribute_mapping(attribute::String)
    for row in 1:size(attribute_map, 1)
        if attribute == attribute_map[row, 1]
            return attribute_map[row, 2]
        end
    end
    println("Fann ekki attribute match fyrir ,$attribute,")
    return attribute
            
end
println("...")
function subfund_mapping(subfund_name::String)
    for row in 1:size(subfund_map, 1)
        if subfund_name == subfund_map[row, 1]
            return subfund_map[row, 2]
        end
    end
    println("Fann ekki subfund match fyrir ,$subfund_name,")
    return subfund_name
end
println("..")
function insert_subfunds_SF(sheet::DataFrame)
    # Loop through each row in the value matrix and insert the data into the PostgreSQL database
    for row in 2:size(sheet, 1)
        for col in 1:size(sheet, 2)
            fund_name    = sheet[row, 1]
            short_name   = sheet[row, 2]
            subfund_name = subfund_mapping(sheet[row, 3])
            
            fundtype     = sheet[row, 4]
            
            reg_string = ""
            fund_id = -100
            for reg in regex_key
                if !isnothing(match(reg, fund_name))
                    reg_string = string(reg)[3:end-1]
                    result = execute(conn, "SELECT fund_id FROM fund WHERE regex_keyword = '$reg_string' or short_name = '$fund_name' or fund_name = '$fund_name'") |> DataFrame
                    if size(result,1) == 0
                        error("Fann ekki fund_id í gagnagrunni fyrir : $reg_string, fund_name er : $fund_name")
                    end
                    fund_id = result[1,1]
                    break
                end
            end
            
            if fund_id == -100
                error("no fund_id found for : $fund_name")
            else
                result = execute(conn, "SELECT subfund_id FROM subfund WHERE fund_id = $fund_id and subfund_name = '$subfund_name'") |> DataFrame
                if size(result,1) == 0
                    sql = "INSERT INTO public.subfund(fund_id, short_name, subfund_type, regex_keyword, subfund_name) VALUES ($fund_id, '$short_name', '$fundtype', '$reg_string', '$subfund_name');"
                    execute(conn, sql)
                end
            end
        end
    end
end
println(".")
function insert_subfunds_funddata(sheet::DataFrame, fund_type::String)
    # Loop through each row in the value matrix and insert the data into the PostgreSQL database
    for col in 6:size(sheet, 2)
        fund_name = sheet[1, col]
        subfund_name = subfund_mapping(sheet[2, col])
        
        short_name   = ""
        fundtype     = fund_type

        reg_string = ""
        fund_id = -100
        for reg in regex_key
            if !isnothing(match(reg, fund_name))
                reg_string = string(reg)[3:end-1]
                result = execute(conn, "SELECT fund_id FROM fund WHERE regex_keyword = '$reg_string' or short_name = '$fund_name' or fund_name = '$fund_name'") |> DataFrame
                if size(result,1) == 0
                    error("Fann ekki fund_id í gagnagrunni fyrir : $reg_string, fund_name er : $fund_name")
                end
                fund_id = result[1,1]
                break
            end
        end

        if fund_id == -100
            error("no fund_id found for : $fund_name")
        else
            result = execute(conn, "SELECT short_name FROM fund WHERE fund_id = $fund_id") |> DataFrame
            short_name = result[1,1]

            result = execute(conn, "SELECT subfund_id FROM subfund WHERE fund_id = $fund_id and subfund_name = '$subfund_name'") |> DataFrame
            if size(result,1) == 0
                
                sql = "INSERT INTO public.subfund(fund_id, short_name, subfund_type, regex_keyword, subfund_name) VALUES ($fund_id, '$short_name', $fundtype, '$reg_string', '$subfund_name');"
                execute(conn, sql)
            end
        end
    end
end
println("..")
# Loop through each Excel file in the directory
for year in 1997:2021
    println("...")
    println("Erum að fara í $year")
    
    filename = "Arsreikningar-lifeyrissjoda_$year.xlsx"
    filepath = joinpath(dir_path, filename)
    
    funddata_sam = DataFrame(XLSX.readtable(filepath, "funddata sam", header = false))
    funddata_ser = DataFrame(XLSX.readtable(filepath, "funddata ser", header = false))
    funddata_hluti_3 = DataFrame(XLSX.readtable(filepath, "funddata hluti 3", header = false))
    
    SF = DataFrame(XLSX.readtable(filepath, "SF", header = false))
    
    insert_subfunds_SF(SF)
    println("subfunds insert er buid")
    insert_subfunds_funddata(funddata_sam,"'Samtrygging'")
    println("Samtrygging check")
    insert_subfunds_funddata(funddata_ser,"'Séreign'")
    println("Séreign check")
    insert_subfunds_funddata(funddata_hluti_3,"NULL")
    println("Hluti 3 check")
    
    
    
    
    insert_standardform(funddata_sam, year)
    println("sam er buid að inserta inní funddata")
    insert_standardform(funddata_ser, year)
    println("ser er buid að inserta inní funddata")
    insert_standardform(funddata_hluti_3, year)
    println("hluti 3 er buid að inserta inní funddata")
end
println("Fjöldi null sem ég fann = $nullcounter")


.
..
...
..
.
..
...
..
.
..
...
Erum að fara í 1997
subfunds insert er buid
Samtrygging check
Séreign check
Hluti 3 check
sam er buid að inserta inní funddata
ser er buid að inserta inní funddata
hluti 3 er buid að inserta inní funddata
...
Erum að fara í 1998
subfunds insert er buid
Samtrygging check
Séreign check
Hluti 3 check
sam er buid að inserta inní funddata
ser er buid að inserta inní funddata
hluti 3 er buid að inserta inní funddata
...
Erum að fara í 1999
subfunds insert er buid
Samtrygging check
Séreign check
Hluti 3 check
sam er buid að inserta inní funddata
ser er buid að inserta inní funddata
hluti 3 er buid að inserta inní funddata
...
Erum að fara í 2000
subfunds insert er buid
Samtrygging check
Séreign check
Hluti 3 check
sam er buid að inserta inní funddata
ser er buid að inserta inní funddata
hluti 3 er buid að inserta inní funddata
...
Erum að fara í 2001
subfunds insert er buid
Samtrygging check
Séreign check
Hluti 3 check
sam er buid að inserta inní funddata
ser 

In [55]:
using XLSX
using LibPQ
using DataFrames

regex_key = [r".*(A|a)llianz.*",r".*Almenni.*",r".*VÍB.*",r".*(A|a)rion.*",r"(b|B)ayern",r".*Birta.*",r".*(B|b)rú.*",r"Eftirlaunasj.*\sLandsb(anka|\.) (og|&) Seðlabanka",r"Eftirlaunasj.*(Sláturf(él|.)).*(Suðurlands).*",r"Eftirlaunasj(\.|óður)(\s|)(s|S)tarfsm(\.|anna|m\.)(\s|)Íslandsb(\.|anka)(\s(h|H)f\.|).*",r".*Glitn.*",r"Eftirlaunasj.*(Hafnarfj).*(kaupst\.|kaupstaðar|arðk\.|arðark\.)",r"Eftirlaunasj.*(|\n)(Olí|OLÍ).*",r"(EFÍA|Eftir(-|)launa(-|-\s|)sj(\.|\s|óður)(\s|\s\s|)(atvinnuflugmanna|FÍA))",r"Eftirlaunasj.*Reykjanes(-|-\s|)bæjar",r".*Eftirlaunasj.*(\s|)slökkvil.* á(\s|)Keflav.*",r"Eftirl.*\s*(starfsm|stm).*\s*(Ú|ú)tvegsb.*\s*Ísl.*",r".*Festa.*",r".*Frjálsi.*",r".*(G|g)ildi.*",r".*(Í|í)slandsbanki.*",r".*Íslenski.*",r"(Kjölur lífeyris).*",r".*(K|k)vika.*",r".*(L|l)andsbankinn.*",r"Lífeyris(-sjóður|sjóður|sj)(\s|\.|\.\s)(l|L)ækna.*",r"Lífeyris(-|)(sj\.|sjóður )(\n|\s|)Suðurlands",r"(Lífeyris).*(sjóðurinn).*(Skjöldur)",r".*Eining.*",r".*Hlíf.*",r".*(Akranes(-|-\s|)kaupst).*",r"(L|l)ífeyrissj.*(\.|\s)(Bolungar).*",r"(Lífeyris).*(sj).*(Eimskipa).*",r"Lífeyrissj.*ísl.*stjst.* á Kefl.{0,1}v.{0,4}flugv.*",r".*((\.|\s)Nes(-|)kaup).*(\.|staðar)",r".*(Húsa(-|-\s|)víkur(k|k\.|-|))(\.|aupstaðar|bæjar|).*",r".*Lífeyris(-|)(sj\.|sjóður )(\s|)(stm\.|starfsm\.|starfsmanna)\sR(eykjavíkur|vík\.)(-|-\s|\s|)apóteks.*",r"Lífeyrissj(\.|óður\s)(\s|)(stm|starfsmanna|starfsm\.)(\.\s|\s|)Vestm.*eyja(\s|b\.|bæjar|-bæjar)",r".*(A|a)rkitekta.*(T|t)æknifr.*",r".*K\.E\.A\..*",r".*Líf.*(V|v)erkal.*fél\.(\sá\s|\s|)Norðurl\. v.*",r"Lífeyrissj.* alþingis(-\s|)manna.*",r"Lífeyris(-|-\s|)sjóður Arkitekta",r"(L|l)ífeyris(-|)sj(\.|óður )(\s|)(Austur(-|-\s|)lands).*",r".*(B|b)ænda.*",r".*Lífeyrissj.*(b|B)anka(-|-\s|)m.*",r".*(B|b)laða(-\s|)manna.*",r"(l|L)ífeyrissj.*(F|f)élags.*ísl.*leikara",r"(L|l)ífeyrissj.*(\.|\s)(Flugvirkja(-f|fé|f.)).*",r".*(L|l)íf.+(H|h)júkrunar(-|-\s|)fræðinga.*",r"Lífeyrissj.*Iðnaðarm.*Suðurnesja",r".*(L|l)eigubifr.*",r"(L|l)ífeyrissj.*(\.|\s).*(Mjólkur).*",r"(L|l)ífeyrissj.*(\.|\s)(Norður(-|-\s|)(lands|))",r"Lífeyrissj.* ráðherra",r"Lífeyris(-|)sj(\.|óður\s)(\s|)Rang(\.|æinga)",r".*(S|s)jómanna.*",r".*(S|s)jóvá.*",r"Lífeyrissj(óður|\.\s|\.)(\s|\.|)(stm\.|starfsm)(anna\s|\s|\.\s)Akur(-|-\s|)eyrar(b\.|bæjar)",r".*(B|b)únaðar.*",r".*Lífeyrissj(óður|\.)(\s|)(stm\.|starfsm\.|starfsmanna)\sKópa(-|-\s|)vogs(b\.|bæjar|kaupst).*",r".*Lífeyris(sjóður|-sjóður|sj\.)(\s|\.|)((S|s)tm\.\s|(S|s)tarfsmanna|(S|s)tarfsm\.)(\s|)Reykjav(íkurb\.|b\.|íkurborgar|\.).*",r"(Lífeyrissj.*(starfsm(\.|anna)|\.(\s|)st\.|\.\sst\.m\.|stm\.)\s(R|r)íkisins|.*LSR.*)",r"Lífeyriss(s|)j(óður|\.)(\s|)(s|S)t.*(\s|\.|)(s|S)veitarfél(\.|aga)",r".*(Áburðarv(.|e)).*",r"Lífeyrissj(\.|óður)(\s|)Suður(-|-\s|)nesja",r"Lífeyrissj.*Tæknifr.* Íslands",r".*(Tannl|tannl).*",r".*Grindavík.*",r"Lífeyrissj.*verkal.* á Suðurl(andi|\.)",r".*(Lífeyrissj).*((V|v)erk.*)fr.*",r".*((L|l)íf[a-z. óð]*Versl|(L|l)íf[a-z. óð]*versl).*",r"Lífeyrissj(óður|\.)((\s|)Vest(-\s|-\s\s|-|)firðinga)",r".*Lífeyrissj(\.|óður)(\s|)Vestm(\.|(anna(-|-\s|)eyja)).*",r".*Lífeyris(-|)(sj\.|sjóður)(\s|)Vestur(-|-\s|)lands.*",r".*Framsýn.*",r"(L|l)ífey.*(\.|\s)(Lífiðn)+",r".*Lífsval.*",r".*(L|l)ífsverk.*",r"Sameinaði(\s|)lífeyris(-|-\s|)sjóðurinn",r".*(S|s)amvinnu(-|).*",r".*Séreigna.*lífeyris.*sjóðurinn.*",r"(S|s)öfnunarsj(\.\s|\.|óður\s)(lífeyris(-|)réttinda|)",r".*Stafir.*",r".*(S|s)tapi.*",r".*Trygginga.*sjóður.*lækna.*",]

file = open("login_for_sql.txt", "r")
username = chomp(readline(file))
password = chomp(readline(file))
close(file)

conn = LibPQ.Connection("dbname=Symmetria user=$username password=$password")

action_type = ["A", "M", "Q"]
description = ["Active", "Merge", "Quit"]

execute(conn, "BEGIN;")

LibPQ.load!(
    (action_type = action_type, description = description),
    conn,
    "INSERT INTO actions (action_type,description) VALUES (\$1,\$2);",
)
result = execute(conn, "COMMIT;") |> DataFrame 

function read_column(sheet, firstrow, col)
    res = []
    row = firstrow
    while !ismissing(sheet[row,col])
        push!(res,sheet[row,col])
        row+=1
    end 
    return res
end

function rawdata_history(xf)
    sheet_name = "EST"
    sheet = xf[sheet_name]

    firstrow = 2
    col = 1
    fundname = read_column(sheet, firstrow, col)
    col = 2
    founded = read_column(sheet, firstrow, col)
    col = 3
    action_type = read_column(sheet, firstrow, col)
    col = 4
    date = read_column(sheet, firstrow, col)
    col = 7
    source = read_column(sheet, firstrow, col)

    rawdata = DataFrame(fundname = fundname, founded = founded, 
            action_type = action_type, date = date, source = source)

    return rawdata
end

function rawdata_merge(xf)
    sheet_name = "Merge"
    sheet = xf[sheet_name]

    firstrow = 1
    col = 1
    fundname = read_column(sheet, firstrow, col)
    col = 2
    merged_fundname = read_column(sheet, firstrow, col)

    rawdata = DataFrame(fundname = fundname, merged_fundname = merged_fundname)
    return rawdata
end


path = "skjölin frá Birgi/Files/Merged_funds.xlsx"
xf = XLSX.readxlsx(path)
fund_history = rawdata_history(xf)

me = XLSX.readxlsx(path)
merge_history = rawdata_merge(me)

for row in eachrow(fund_history)
    reg_string = ""
    fund_id = -100
    
    for reg in regex_key
        if !isnothing(match(reg, row.fundname))
            reg_string = string(reg)[3:end-1]
            result = execute(conn, "SELECT fund_id FROM fund WHERE regex_keyword = '$reg_string' or fund_name = '$(row.fundname)'") |> DataFrame
            if size(result,1) == 0
                error("Fann ekki fund_id í gagnagrunni fyrir : $reg_string, fund_name er : $(row.fundname)")
            end
            fund_id = result[1,1]
            break
        end
    end

    if fund_id == -100
        error("no fund_id found for: $(row.fundname)")
    else
        if row.founded == "NULL"
            founded_str = "NULL"
        else
            founded_str = "'$(row.founded)'"
        end

        if row.source == "NULL"
            source_str = "NULL"
        else
            source_str = "'$(row.source)'"
        end
        destfund_id = -100
        destfund_name = "-"
        if row.action_type == "A"
            destfund_id = fund_id
        elseif row.action_type == "M"
            fundname = row.fundname
            for row in 1:size(merge_history, 1)
                if fundname == merge_history[row, 1]
                    destfund_name = merge_history[row, 2]
                    for reg in regex_key
                        if !isnothing(match(reg,destfund_name))
                            reg_string = string(reg)[3:end-1]
                            result = execute(conn, "SELECT fund_id FROM fund WHERE regex_keyword = '$reg_string' or fund_name = '$(destfund_name)'") |> DataFrame
                            if size(result,1) == 0
                                error("Fann ekki fund_id í gagnagrunni fyrir : $reg_string, fund_name er : $(destfund_name)")
                            end
                            destfund_id = result[1,1]
                            break
                        end
                    end
                    break
                end
            end
            
        else
            destfund_id = "NULL"
        end

        if destfund_id == -100
            error("fundum ekki destination fund, $fundname, $destfund_name")
        end

        result = execute(conn, "SELECT * FROM fundhistory WHERE origin_id = $(fund_id) AND date = '$(row.date)' AND action_type = '$(row.action_type)' AND founded = $founded_str AND source = $source_str") |> DataFrame
        if size(result,1) == 0
            sql = "INSERT INTO public.fundhistory(origin_id, destination_id, date, action_type, founded, source) VALUES ($(fund_id), $destfund_id, '$(row.date)', '$(row.action_type)', $founded_str, $source_str);"
            execute(conn, sql)
        end
    end
end

println("Fundhistory komið í gagnagrunninn")



Fundhistory komið í gagnagrunninn
